In [6]:
from pyforest import *
lazy_imports()
import warnings
warnings.filterwarnings("ignore")

In [9]:
df = pd.read_csv("Final_data.csv")
df.head()

<IPython.core.display.Javascript object>

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,source,Outlet_Years,Item_Visible_Avg,Item_Fat_Content_1,Item_Fat_Content_2,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_1,Outlet_Size_2,New_Item_Type_1,New_Item_Type_2,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,3735.1380,train,10,0.931078,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,443.4228,train,1,0.933420,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,2097.2700,train,10,0.960069,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,train,11,0.000000,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.000000,Household,53.8614,OUT013,1987,994.7052,train,22,0.000000,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0


In [10]:
# Dropping Item Type and Establishment year as we have extracted separate columns with more accurate information

print(f"Dataset size before dropping : {df.shape}")
df = df.drop(['Item_Type','Outlet_Establishment_Year'], axis = 1)
print(f"Dataset size before dropping : {df.shape}")

Dataset size before dropping : (14204, 31)
Dataset size before dropping : (14204, 29)


In [13]:
df[df['source'] == 'train']

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Item_Outlet_Sales,source,Outlet_Years,Item_Visible_Avg,Item_Fat_Content_1,Item_Fat_Content_2,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_1,Outlet_Size_2,New_Item_Type_1,New_Item_Type_2,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.300,0.016047,249.8092,OUT049,3735.1380,train,10,0.931078,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.920,0.019278,48.2692,OUT018,443.4228,train,1,0.933420,0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.500,0.016760,141.6180,OUT049,2097.2700,train,10,0.960069,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.200,0.000000,182.0950,OUT010,732.3800,train,11,0.000000,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NCD19,8.930,0.000000,53.8614,OUT013,994.7052,train,22,0.000000,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,0.056783,214.5218,OUT013,2778.3834,train,22,0.874001,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
8519,FDS36,8.380,0.046982,108.1570,OUT045,549.2850,train,7,1.001096,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0
8520,NCJ29,10.600,0.035186,85.1224,OUT035,1193.1136,train,5,0.998881,1,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0
8521,FDN46,7.210,0.145221,103.1332,OUT018,1845.5976,train,1,1.041620,0,1,0,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0


In [14]:
# Let's divided the dataset to Train and Test
train = df[df['source'] == 'train']
test = df[df['source'] == 'test']

X_train = train.drop(['Item_Outlet_Sales', 'Item_Identifier', 'Outlet_Identifier', 'source'], axis = 1)
y_train = train['Item_Outlet_Sales']
X_test = test.drop(['Item_Outlet_Sales', 'Item_Identifier', 'Outlet_Identifier', 'source'], axis = 1)

#### Our Problem Statement to determine the Sales based on Store and Items features as input

#### Linear Regression

In [16]:
# Creating Object of Linear Regression Model and Fit it to determine the Cooeficients, Slope and best fit line equation

lr = LinearRegression(normalize= True)
lr.fit(X_train, y_train)

<IPython.core.display.Javascript object>

LinearRegression(normalize=True)

In [17]:
lr_pred = lr.predict(X_test)
lr_pred

array([1844.08821509, 1569.9023178 , 1871.14116321, ..., 1805.82428049,
       3573.54521621, 1263.41875663])

In [19]:
# Let's see the accuracy of our model in our training data

lr_accuracy = round(lr.score(X_train, y_train)*100,2)
print(f"Linear Regression Accuracy Score in Training Data : {lr_accuracy}")

Linear Regression Accuracy Score in Training Data : 56.35


#### Linear Regression Model is not performing well enough

#### Decision Tree Regressor

In [22]:
# Importing Decision Tree Regressor Package
from sklearn.tree import DecisionTreeRegressor

# Training our Model object with train data
# Max Depth and Min Samples Leaf are hyperparameters that are used to avoid overfitting our training model
dec_reg = DecisionTreeRegressor(max_depth = 12, min_samples_leaf = 60)
dec_reg.fit(X_train, y_train)

# Make predictions using the trained model created
dec_pred = dec_reg.predict(X_test)

# Checking on the Accuracy on our training data
tree_accuracy = round(dec_reg.score(X_train, y_train)*100,2)
print(f"Decision Tree Regressor Accuracy Score in Training Data : {tree_accuracy}")

Decision Tree Regressor Accuracy Score in Training Data : 61.39


####  We see slight increase in our model
#### Will use Random Forest Ensemble Method

In [23]:
# Training our Model object with train data
# n_estimators denotes the amount of Decision tress that will be created in parellel for Model training
# Max Depth and Min Samples Leaf are hyperparameters that are used to avoid overfitting our training model

rf = RandomForestRegressor(n_estimators= 500, max_depth= 6, min_samples_leaf= 20)
rf.fit(X_train, y_train)

# Make predictions using the trained model created
rf_pred = rf.predict(X_test)

# Checking on the Accuracy on our training data
rf_accuracy = round(rf.score(X_train, y_train)*100,2)
print(f"Random Forest Regressor Accuracy Score in Training Data : {rf_accuracy}")

<IPython.core.display.Javascript object>

Random Forest Regressor Accuracy Score in Training Data : 60.49


#### Decision Trees and Random Forest are performing similar with our training data
#### Let's use XG Boost Regressor

In [26]:
# Importing XG Boost Regressor
from xgboost import XGBRegressor

# Training our Model object with train data
# n_estimators denotes the amount of Decision tress that will be created in parellel for Model training
# learning rate denotes the step taking by algorithm towards Global Optima

xg = XGBRegressor(n_estimators= 1000, learning_rate= 0.05)
xg.fit(X_train, y_train)

# Make predictions using the trained model created
xg_pred = xg.predict(X_test)

# Checking on the Accuracy on our training data
xg_accuracy = round(xg.score(X_train, y_train)*100,2)
print(f"XG Boost Regressor Accuracy Score in Training Data : {xg_accuracy}")

[01:03:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XG Boost Regressor Accuracy Score in Training Data : 69.82
